In [18]:
using BenchmarkTools

include("../src/Cosmojuly.jl")
import .Cosmojuly as cj

In [19]:
#cj.cumulative_progenitors(9.998215795129126e11/2.0, 9.998215795129126e11, 1e+8)
#cj.draw_mass_with_restrictions(9.998215795129126e11, 1e+8)
#cj.Δω_approx(1e+12, 1e+8)
#cj.redshift_array(0, 0.5*cj.Δω_approx(1e+12, 1e+8))
cj.subhalo_mass_function(1e+12, 1e+6)

0.015490629722080484 48 0.4990509156928255


0.030790368137786572 109 0.49781184534186007


0.04590704245129018 162 0.49658205865215793


0.06086309102647647 205 0.24772670207500674


0.07564821752384887 229 0.24721991123779086


...

4.069086872739717 423 0.0002028548470925793


4.076782973250374 423 0.00020251634337730722


4.084479073761031 423 0.00020217836315444932


4.092175174271688 423 0.00020184090565920755


InterruptException: InterruptException: